In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling1D, Conv1D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import to_categorical

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

import sys

import numpy as np

Using TensorFlow backend.


In [2]:
noise_len = 20
cutoff = 3

In [3]:
def process_data():
    X_train = [
        "Building a bike",
        "Building a robot",
        "Building a wall",
        "Writing a program",
        "Writing an article",
        "Writing a play",
        "Learning to draw",
        "Learning to paint",
        "Learning to cook",
        "Learning Klingon",
        "Learning Java",
        "Exploring math",
        "Exploring art",
        "Exploring mindfulness"
    ]
    
    # for the purpose of this model, I am disregarding the capitalization of titles
    X_train = [i.lower() for i in X_train]
    
    # changes the token pattern to include single character words
    word_vector = CountVectorizer(token_pattern=u'(?u)\\b\\w+\\b')
    
    term_freq = word_vector.fit_transform(X_train)
    word_index = word_vector.vocabulary_
    word_index["PADDING"] = len(word_vector.vocabulary_)
    reverse_word_index = {value: key for key, value in word_index.items()}
    
    # dimensions should be 14 (number of titles/data points) by 3 (words/channels) by 22 (vocabulary/classes)
    
    def split_words(string):
        return string.split(" ")

    def pad_data(data, cutoff):
        for i in data:
            while len(i) < cutoff:
                i.append("PADDING")

    def index_words(word_list, word_index):
        return [word_index[i] for i in word_list]

    X_train = list(map(split_words, X_train))

    pad_data(X_train, cutoff)

    X_train = [index_words(i, word_index) for i in X_train]
#     return to_categorical(X_train, num_classes=22)#[i] for i in range(len(X_train)))
    return(
        np.array([np.transpose(to_categorical(X_train, num_classes=22)[i]) for i in range(len(X_train))]),
        reverse_word_index
    )

In [4]:
X_train, vocab = process_data()
print(X_train)

[[[0. 1. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 1.]
  [1. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 1. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [1. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 1.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 1. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [1. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 1.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 1. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 

In [5]:
def build_discriminator():
    '''
    Put together a CNN that will return a single confidence output.
    
    returns: the model object
    '''
    
    # dimensions: rows = list of words (22), columns/channels = # of words per title (3)
    
    model = Sequential()
    model.add(Conv1D(32, kernel_size=3, strides=2, input_shape=(22, 3), padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv1D(64, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv1D(128, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv1D(256, kernel_size=3, strides=1, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    return model

In [6]:
def build_generator():
    '''
    Put together a model that takes in one-dimensional noise and outputs two-dimensional
    data representing a three-word phrase.
    
    returns: the model object
    '''
    
    # note to self: trailing commas are used for single-element tuples
    noise_shape = (noise_len,)

    model = Sequential()

    model.add(Dense(22 * 3, activation="relu", input_shape=noise_shape))
    model.add(Reshape((22, 3)))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv1D(128, kernel_size=3, padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(momentum=0.8)) 
    model.add(Conv1D(64, kernel_size=3, padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv1D(3, kernel_size=3, padding="same"))
    model.add(Activation("tanh"))

    return model

In [7]:
def build_combined():
    '''
    Puts together a model that combines the discriminator and generator models.
    
    returns: the generator, discriminator, and combined model objects
    '''
    
    optimizer = Adam(0.0002, 0.5)

    # Build and compile the discriminator
    discriminator = build_discriminator()
    discriminator.compile(loss='binary_crossentropy', 
                          optimizer=optimizer,
                          metrics=['accuracy'])


    # Build and compile the generator
    generator = build_generator()
    generator.compile(loss='binary_crossentropy', optimizer=optimizer)

    # The generator takes noise as input and generates images
    noise = Input(shape=(noise_len,))
    title = generator(noise)
    
    
    # For the combined model we will only train the generator
    discriminator.trainable = False

    # The discriminator takes generated images as input and determines validity
    valid = discriminator(title)

    # The combined model  (stacked generator and discriminator) takes
    # noise as input => generates images => determines validity 
    combined = Model(inputs=noise, outputs=valid)
    combined.compile(loss='binary_crossentropy', optimizer=optimizer)
    return generator, discriminator, combined

In [8]:
def save_titles(generator, epoch, vocab):
    '''
    Has the generator create and save new Quest titles, so God help us all.
    
    inputs:
        generator: the generator model object returned by build_combined
        epoch: the epoch number (but can be anything that can be represented as a string)
        vocab: the mapping of numbers to words
    
    returns: None
    '''
    titles = 1
    
    noise = np.random.normal(0, 1, (titles, noise_len))
    gen_title = generator.predict(noise)
    
    # chooses the word with the highest weight
    gen_title = [np.argmax([j[i] for j in gen_title[0]]) for i in range(3)]
    
    # map words to numbers
    gen_title = " ".join([vocab[i] for i in gen_title])
    
    file=open('titles/titles_{}.txt'.format(epoch),"w+")
    file.write(gen_title)
    print(gen_title)
    file.close()

In [9]:
def train(generator, discriminator, combined, data, vocab, epochs, batch_size=128, save_interval=50):
    '''
    Trains all model objects
    
    generator: the generator model object returned by build_combined
    discriminator: the discriminator model object returned by build_combined
    combined: the combined model object returned by build_combined
    epochs: integer, the number of epochs to train for
    batch_size: integer, the number of training samples to use at a time
    save_interval: integer, will generate and save images when the current epoch % save_interval is 0
    
    returns: None
    '''

    # Load the dataset
    X_train = data

    half_batch = int(batch_size / 2)

    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random half batch
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        titles = X_train[idx]

        # Sample noise and generate a half batch of new images
        noise = np.random.normal(0, 1, (half_batch, noise_len))
        gen_titles = generator.predict(noise)

        # Train the discriminator (real classified as ones and generated as zeros)
        d_loss_real = discriminator.train_on_batch(titles, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_titles, np.zeros((half_batch, 1)))

        # ---------------------
        #  Train Generator
        # ---------------------

        noise = np.random.normal(0, 1, (batch_size, noise_len))
        # Train the generator (wants discriminator to mistake images as real)
        g_loss = combined.train_on_batch(noise, np.ones((batch_size, 1)))
           
        # If at save interval => save generated image samples and plot progress
        if epoch % save_interval == 0:
            # Plot the progress
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            print ("{} [D loss: {}, acc.: {:.2%}] [G loss: {}]".format(epoch, d_loss[0], d_loss[1], g_loss))
            save_titles(generator, epoch, vocab)

In [10]:
generator, discriminator, combined = build_combined()

In [11]:
train(generator, discriminator, combined, X_train, vocab, epochs=4001, batch_size=32, save_interval=50)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.8421581983566284, acc.: 40.62%] [G loss: 0.6872787475585938]
klingon cook building
50 [D loss: 0.35807278752326965, acc.: 78.12%] [G loss: 1.859682559967041]
building building robot
100 [D loss: 0.25104019045829773, acc.: 96.88%] [G loss: 2.043210744857788]
mindfulness play to
150 [D loss: 0.0837550163269043, acc.: 100.00%] [G loss: 2.999795436859131]
to learning learning
200 [D loss: 0.11396276950836182, acc.: 96.88%] [G loss: 3.410893678665161]
math program paint
250 [D loss: 0.08014360815286636, acc.: 100.00%] [G loss: 4.34560489654541]
writing math cook
300 [D loss: 0.11298981308937073, acc.: 96.88%] [G loss: 5.073624610900879]
exploring a article
350 [D loss: 0.0650494173169136, acc.: 100.00%] [G loss: 5.997878551483154]
learning klingon klingon
400 [D loss: 0.1469355821609497, acc.: 93.75%] [G loss: 5.929841041564941]
PADDING to robot
450 [D loss: 0.0684782937169075, acc.: 100.00%] [G loss: 5.444269180297852]
writing a bike
500 [D loss: 0.04744041711091995, acc.: 96.